# Demo: Serenity Factor Risk Model API

The heart of the Serenity API at this time are the risk functions. This notebook demonstrates how you can take a portfolio
assembled out of assets from our security master and compute a market risk attribution as of a given date. It also shows
how you can take that output and pivot the risk by asset, sector and factor to get a deeper understanding of the sources
of risk in your portfolio.

In [ ]:
%%capture --no-stderr --no-display
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from serenity_sdk.widgets import ConnectWidget

# if you want to auto-connect, set this environment variable to your desired default
connect_widget = ConnectWidget(os.getenv('SERENITY_CONFIG_ID', None))

In [ ]:
import datetime

import pandas as pd
import seaborn as sns

# create an alias to the api
api = connect_widget.get_api()

To run a risk attribution using the Serenity Factor Risk Model (SFRM) you first need to construct a portfolio. 
You can use a number of different symbologies, but for purposes of illustration we will use native blockchain symbol.

In [ ]:
asset_master = api.refdata().load_asset_master()

portfolio_raw = {
    'ADA': 1000000,
    'BTC': 100,
    'ETH': 1000,
    'XRP': 2000000,
    'ALGO': 1500000,
    'SOL': 10000,
    'DOT': 50000
}
portfolio = asset_master.create_portfolio(portfolio_raw, symbology='NATIVE')

The next thing we need is a model configuration. Let's choose the medium-time horizon configuration for SFRM, currently the only one supported (and the default):

In [ ]:
from serenity_sdk.types.common import CalculationContext

model_short_name = 'risk.factor.ff.beta.mt.v1_6'  
model_meta = api.model().load_model_metadata(datetime.date.today())
model_config_id = model_meta.get_model_configuration_id(model_short_name)

We can now run risk attribution. Past dates not yet cached may take longer to run (still typically less than 10 seconds):

In [ ]:
from datetime import date

# construct the input parameters for risk attribution
ctx = CalculationContext(as_of_date=date(2021, 7, 1), model_config_id=model_config_id)

result = api.risk().compute_risk_attrib(ctx, portfolio)

The optional tables helper class lets you convert risk attribution results to Pandas DataFrames:

In [ ]:
from serenity_sdk.renderers.table import FactorRiskTables

tables = FactorRiskTables(result)

You can summarize the risk of the whole portfolio:

In [ ]:
pct_fmt = lambda val: f'{val:,.1%}'

total_risk_df = tables.to_total_risk_data_frame()
total_risk_df.style.format({
  'factorRisk': pct_fmt,
  'specificRisk': pct_fmt,
  'totalRisk': pct_fmt
})

You can view the factor risks at a summary level:

In [ ]:
tables.to_factor_risk_data_frame()

You can also break out risk by sectors, showing absolute and relative factor / specific / total risk at various levels,
allowing you to build a hierarchy:

In [ ]:
tables.to_sector_risk_data_frame()

If you like, you can pivot by sector and factor too:

In [ ]:
tables.to_sector_factor_risk_data_frame()

Or by assets:

In [ ]:
tables.to_asset_risk_data_frame(asset_master)

### Factor outputs

You can get at a number of factor model outputs via the API as well; note if you optionally pass the portfolio in you can get any asset-specific.
matrices only for the subset of assets in the portfolio, as shown.

You can retrieve the asset covariance matrix:

In [ ]:
asset_cov_df = api.risk().get_asset_covariance_matrix(ctx, asset_master, portfolio)
sns.heatmap(asset_cov_df, cmap='YlGnBu')

You can also get the asset residual covariances:

In [ ]:
res_cov_df = api.risk().get_asset_residual_covariance_matrix(ctx, asset_master, portfolio)
res_cov_df

You can get the factor returns:

In [ ]:
api.risk().get_factor_returns(ctx)

If you want to see the long/short portfolios in each factor index, you can get that too:

In [ ]:
from uuid import UUID

size_factor_pf = api.risk().get_factor_portfolios(ctx)['size']
asset_positions = size_factor_pf.to_asset_positions()
indexcomps = [{'symbol': asset_master.get_symbol_by_id(UUID(position['assetId'])),
               'quantity': position['quantity']} for position in asset_positions]
df = pd.DataFrame(indexcomps)
df.sort_values(['quantity', 'symbol'], inplace=True)
df.reset_index(drop=True)

You can get the factor correlations:

In [ ]:
factor_corr_df = api.risk().get_factor_correlation_matrix(ctx)
display(factor_corr_df)

And you can get the factor covariances:

In [ ]:
factor_cov_df = api.risk().get_factor_covariance_matrix(ctx)
display(factor_cov_df)

Finally, you can get the factor loadings (exposures) for the entire asset universe:

In [ ]:
exposures_df = api.risk().get_asset_factor_exposures(ctx, asset_master, portfolio)
display(exposures_df)